In [1]:
import openai
import nest_asyncio
nest_asyncio.apply()

In [2]:
openai.api_key = 'Your API KEY'

## Setting Up the Query Tools

In [3]:
from utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools('metagpt.pdf', 'metagpt')

## setup Function Calling Agent

In [4]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model='gpt-3.5-turbo', temperature=0)

In [5]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True
)

agent = AgentRunner(agent_worker)

In [7]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT,"
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT,and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The roles in MetaGPT include the Product Manager responsible for creating the Product Requirement Document and competitive analysis, the Architect who designs system architecture and technical specifications, the Project Manager who breaks down tasks and manages the task list, the Engineer who implements code based on specifications, and the QA Engineer who generates unit tests and ensures software quality. Each agent's specialized role contributes to the collaborative software development process in MetaGPT.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents communicate with each other in MetaGPT"}
=== Function Output ===
Agents in MetaGPT communicate with each other through structured commu

In [8]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and a self-generated SoftwareDev dataset.
=== LLM Response ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and a self-generated SoftwareDev dataset.
